# Exercise 3

Check if ffmpeg is installed, if not install it using [Homebrew](https://brew.sh/):

In [2]:
exist = !which ffmpeg
if not exist:
    !brew install ffmpeg

!pip install ffmpeg-python

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached future-0.18.3.tar.gz (840 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492025 sha256=069b52a27b345121ea363c906dd237f824f4779ab20bd2d63b80376e2bf40a4c
  Stored in directory: /Users/dima/Library/Caches/pip/wheels/da/19/ca/9d8c44cd311a955509d7e13da3f0bea42400c469ef825b580b
Successfully built future


In [32]:
import glob
import ffmpeg
from math import gcd 

## Helper functions

In [36]:
def calculate_aspect_ratio(width:int, height:int) -> str:
    """Calculate the video aspect ratio from width and height."""
    
    # Find the greatest common divisor of the width and height
    divisor = gcd(width, height)
    
    # Calculate the simplest form of the aspect ratio
    x = width // divisor
    y = height // divisor
    
    # Return a string representation of the aspect ratio
    return f"{x}:{y}"

In [42]:
def bit_rate_to_mbps(bit_rate:int) -> str:
    """Convert a bit rate in bits per second to megabits per second."""
    
    # Convert the bit rate to megabits per second
    mbps = bit_rate / 1000000
    
    # Return a string representation of the megabits per second
    return f"{mbps} Mb/s"

In [46]:
def bit_rate_to_kbps(bit_rate:int) -> str:
    """Convert a bit rate in bits per second to kilobits per second."""
    
    # Convert the bit rate to kilobits per second
    kbps = bit_rate / 1000
    
    # Return a string representation of the kilobits per second
    return f"{kbps} Kb/s"

## Process files

In [49]:
for file in glob.glob("Exercise3_Films/*"):
    probe_data = ffmpeg.probe(file)
    print("file: " + file)

    # extract metadata about the video stream
    video_stream = next((stream for stream in probe_data['streams'] if stream['codec_type'] == 'video'), None)

    # extract metadata about the audio stream
    audio_strem = next((stream for stream in probe_data['streams'] if stream['codec_type'] == 'audio'), None)

    # check if the container format is mp4
    if not probe_data['format']['format_name'] == "mov,mp4,m4a,3gp,3g2,mj2":
        print("Wrong container format: " + probe_data['format']['format_name'])
    
    # check if the video codec is h264
    if not video_stream['codec_name'] == "h264":
        print("Wrong video codec: " + probe_data['streams'][0]['codec_name'])
    
    # check if the audio codec is aac
    if not audio_strem['codec_name'] == "aac":
        print("Wrong audio codec: " + probe_data['streams'][1]['codec_name'])

    # check if frame rate is 24
    if not video_stream['r_frame_rate'] == "24/1":
        print("Wrong frame rate: " + video_stream['r_frame_rate'])
    
    # check if the video aspect ratio is 16:9
    aspect_ratio = video_stream.get('display_aspect_ratio')
    if not aspect_ratio:
        aspect_ratio = calculate_aspect_ratio(int(video_stream['width']), int(video_stream['height']))

    if not aspect_ratio == "16:9":
        print("Wrong aspect ratio: " + aspect_ratio)
    
    # check if the video resolution is 640x360
    if not int(video_stream['width']) == 640 or not int(video_stream['height']) == 360:
        print(f"Wrong resolution: {video_stream['width']}x{video_stream['height']}")

    # check if the video bitrate is 2 – 5 Mb/s
    video_bit_rate = int(video_stream['bit_rate'])
    if not 2000000 <= video_bit_rate <= 5000000:
        print("Wrong video bitrate: " + bit_rate_to_mbps(video_bit_rate))

    # check if the audio bitrate is up to 256 kb/s
    audio_bit_rate = int(audio_strem['bit_rate'])
    if not audio_bit_rate <= 256000:
        print("Wrong audio bitrate: " + bit_rate_to_kbps(audio_bit_rate))

    # check if the audio channel is stereo
    if not audio_strem['channels'] == 2:
        print("Audio channel is not stereo, a number of channels: " + str(audio_strem['channels']))

    print("\n")


file: Exercise3_Films/Last_man_on_earth_1964.mov
Wrong video codec: prores
Wrong audio codec: pcm_s16le
Wrong frame rate: 24000/1001
Wrong video bitrate: 9.285191 Mb/s
Wrong audio bitrate: 1536.0 Kb/s


file: Exercise3_Films/Voyage_to_the_Planet_of_Prehistoric_Women.mp4
Wrong video codec: hevc
Wrong audio codec: mp3
Wrong frame rate: 30000/1001
Wrong video bitrate: 8.038857 Mb/s
Wrong audio bitrate: 320.0 Kb/s


file: Exercise3_Films/The_Gun_and_the_Pulpit.avi
Wrong container format: avi
Wrong video codec: rawvideo
Wrong audio codec: pcm_s16le
Wrong frame rate: 25/1
Wrong aspect ratio: 180:101
Wrong resolution: 720x404
Wrong video bitrate: 87.438878 Mb/s
Wrong audio bitrate: 1536.0 Kb/s


file: Exercise3_Films/Cosmos_War_of_the_Planets.mp4
Wrong frame rate: 30000/1001
Wrong aspect ratio: 314:177
Wrong resolution: 628x354
Wrong audio bitrate: 317.103 Kb/s


file: Exercise3_Films/The_Hill_Gang_Rides_Again.mp4
Wrong frame rate: 25/1
Wrong video bitrate: 7.53773 Mb/s


